#Analisando os dados nas redes sociais, com menções da marca FGV e derivadas da mesma:

In [77]:
import pandas as pd
import matplotlib as mp
import numpy as np
import nltk
import os
import unicodedata
from collections import Counter
from collections import defaultdict
import string

#Criando um dataframe

In [87]:
df_dicom = pd.read_excel("/home/paulo/Dropbox/Iniciacao_Renato/Arquivos/8_PlanetaY_Ago2014.xls")

In [88]:
#pequenas modificações
df_dicom["CLASS"] = df_dicom.values[:,1]
df_dicom["User"] = df_dicom.values[:,4]

In [89]:
def clean_string(s):
    s = unicodedata.normalize('NFKD', unicode(s)).encode('ascii','ignore')
    delete_symbols = string.punctuation
    delete_words = nltk.corpus.stopwords.words('portuguese')
    string_1 = ''
    string_2 = []
    #limpando os símbolos
    for letter in s:
        if letter not in delete_symbols:
            string_1 += letter
        else:
            string_1 += ' '
    string_1 = string_1.split()
    #limpando as stopwords
    for word in string_1:
        if word not in delete_words:
            string_2.append(word)
    return [word.upper() for word in string_2]

#Treinando o classificador:

In [90]:
def treinar(df,train_percent): #train_percent -> porcentagem de frases que serão usadas pra treinar o classificador
    pos_texts = df[df["CLASS"]=="Positiva"]["Texto"].tolist()
    neg_texts = df[df["CLASS"]=="Negativa"]["Texto"].tolist()
    neut_texts = df[df["CLASS"]=="Neutra"]["Texto"].tolist()
    tam = min(len(pos_texts),len(neg_texts),len(neut_texts))
    
    #limpando os textos
    pos_clean = [clean_string(p) for p in pos_texts[:tam]]
    neg_clean = [clean_string(p) for p in neg_texts[:tam]]
    neut_clean = [clean_string(p) for p in neut_texts[:tam]]
    
    #frases para treino
    a = int(round(train_percent*tam))
    pos_train = pos_clean[:a]
    neg_train = neg_clean[:a]
    neut_train = neut_clean[:a]
    
    #frases para teste
    pos_test = pos_clean[a:tam]
    neg_test = neg_clean[a:tam]
    neut_test = neut_clean[a:tam]
    
    words_count_pos = dict()
    words_count_neg = dict()
    words_count_neut = dict()
    
    for p in neg_clean+neut_clean+pos_clean:
        for word in p:
            words_count_pos[word] = 1
            words_count_neg[word] = 1
            words_count_neut[word] = 1
    for i in range(a):
        p = pos_train[i]
        for word in p:
            words_count_pos[word] += 1
        p = neg_train[i]
        for word in p:
            words_count_neg[word] += 1
        p = neut_train[i]
        for word in p:
            words_count_neut[word] += 1
    return words_count_pos,words_count_neg,words_count_neut,tam,pos_test,neg_test,neut_test

#Alguns fatos:

In [91]:
print "A palavra que mais aparece nas frases positivas é '", words_count_pos.keys()[(words_count_pos.values().index(max(words_count_pos.values())))], "', nas neutras '", words_count_neut.keys()[(words_count_neut.values().index(max(words_count_neut.values())))], "' e nas negativas '", words_count_neg.keys()[(words_count_neg.values().index(max(words_count_neg.values())))], "'."

A palavra que mais aparece nas frases positivas é ' FGV ', nas neutras ' FGV ' e nas negativas ' FGV '.


#Avaliando as probabilidades de cada palavra e frase:

In [92]:
def prob(palavra,words_count_pos,words_count_neg,words_count_neut):
    a = float(words_count_pos[palavra])
    b = float(words_count_neg[palavra])
    c = float(words_count_neut[palavra])
    return [(a/(a+b+c)),(b/(a+b+c)),(c/(a+b+c))]

In [93]:
def eval_phrase(phrase,words_count_pos,words_count_neg,words_count_neut):
    pr = [1/3.,1/3.,1/3.]
    c = ''
    for word in phrase:
        aux = prob(word,words_count_pos,words_count_neg,words_count_neut)
        pr[0] *= aux[0]
        pr[1] *= aux[1]
        pr[2] *= aux[2]
    n = pr.index(max(pr))
    if n == 0: c = "Positiva"
    elif n == 1: c = "Negativa"
    else: c = "Neutra"
    return c

#Criando o classificador:

In [94]:
def classificador1(df,train_percent = 0.9):
    aux = treinar(df,train_percent)
    #contagem das palavras
    words_count_pos = aux[0]
    words_count_neg = aux[1]
    words_count_neut = aux[2]
    #textos para teste
    pos_test = aux[4]
    neg_test = aux[5]
    neut_test = aux[6]
    all_for_test = pos_test + neg_test + neut_test
    
    tam1 = len(pos_test) 
    tam2 = len(neg_test)
    tam3= len(neut_test)
    
    classifications = ["Positiva" for i in range(tam1)] + ["Negativa" for i in range(tam2)] + ["Neutra" for i in range(tam3)]
    class_list = [eval_phrase(p,words_count_pos,words_count_neg,words_count_neut) for p in all_for_test]
    
    counter = 0.
    for i in range(tam1+tam2+tam3):
        if class_list[i] == classifications[i]:
            counter += 1
    print "A porcentagem de acertos do classificador é ", (counter/(tam1+tam2+tam3))

In [95]:
classificador1(df_dicom,0.9)

A porcentagem de acertos do classificador é  0.8
